# LangChain: Agents

## Outline:

* Using built in LangChain tools: DuckDuckGo search and Wikipedia
* Defining your own tools

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings("ignore")

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [2]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

## Built-in LangChain tools

In [ ]:
#!pip install -U wikipedia

In [3]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

In [4]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [5]:
tools = load_tools(tool_names=["llm-math","wikipedia"], llm=llm)

In [6]:
agent = initialize_agent(tools=tools, llm=llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True, verbose = True)

In [7]:
agent("What is the 25% of 300?")



> Entering new AgentExecutor chain...
Question: What is the 25% of 300?
Thought: We can use the calculator tool to find the answer to this math question.
Action:
```
{
  "action": "Calculator",
  "action_input": "25% of 300"
}
```

Observation: Answer: 75.0
Thought:Final Answer: 75.0

> Finished chain.


{'input': 'What is the 25% of 300?', 'output': '75.0'}

## Wikipedia example

In [8]:
question = """Tom M. Mitchell is an American computer scientist 
and the Founders University Professor at Carnegie Mellon 
University (CMU) what book did he write?"""
result = agent(question) 



> Entering new AgentExecutor chain...
Thought: I should use Wikipedia to find out which book Tom M. Mitchell wrote.
Action:
```
{
  "action": "Wikipedia",
  "action_input": "Tom M. Mitchell"
}
```
Observation: Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Sciences, the American Association for the Advancement of Science and a Fellow and past president of the Association for the Advancement of Artificial Intelligence. In October 2018, Mitchell was appointed as t

Thought:I should use Wikipedia to find out the book written by Tom M. Mitchell.
Action:
```
{
  "action": "Wikipedia",
  "action_input": "Machine Learning (book)"
}
```
Observation: Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions. Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.
ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.
Statistics and mathematical optimisation (mathematical programming) methods comprise the 

Thought:Final Answer: The book written by Tom M. Mitchell is "Machine Learning."

> Finished chain.


## Python Agent

In [9]:
agent = create_python_agent(llm=llm, tool=PythonREPLTool(),
    verbose=True
)

In [10]:
customer_list = [
    ["Harrison", "Chase"], 
    ["Lang", "Chain"],
    ["Dolly", "Too"],
    ["Elle", "Elem"], 
    ["Geoff","Fusion"], 
    ["Trance","Former"],
    ["Jen","Ayai"]
]

In [11]:
agent.run(f"""Sort these customers by last name and then first 
name and print the output: {customer_list}""") 



> Entering new AgentExecutor chain...
We can use the `sorted()` function in Python to sort the list of customers based on their last name and then first name.
Action: Python REPL
Action Input: customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]
sorted_customers = sorted(customers, key=lambda x: (x[1], x[0]))
print(sorted_customers)
Observation: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]

Thought:The customers are now sorted by last name and then first name.
Final Answer: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]

> Finished chain.


"[['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]"

#### View detailed outputs of the chains

In [12]:
import langchain
langchain.debug=True
agent.run(f"""Sort these customers by last name and then first 
name and print the output: {customer_list}""") 
langchain.debug=False

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first \nname and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first \nname and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code

## Define your own tool

In [ ]:
#!pip install DateTime

In [13]:
from langchain.agents import tool
from datetime import date

In [14]:
@tool
def time(text: str) -> str:
    """
    Returns todays date, use this for any questions related to 
    knowing todays date. The input should always be an empty 
    string, and this function will always return todays date - 
    any date mathmatics should occur outside this function.
    """
    return str(date.today())

In [15]:
agent = initialize_agent(tools=tools + [time], llm=llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True, verbose = True)

**Note**: 

The agent will sometimes come to the wrong conclusion (agents are a work in progress!). 

If it does, please try running it again.

In [16]:
try:
    result = agent("whats the date today?") 
except: 
    print("exception on external access")



> Entering new AgentExecutor chain...
Thought: I should use the `time` tool to find out today's date.

Action:
```
{
  "action": "time",
  "action_input": ""
}
```


Observation: 2025-12-27
Thought:Final Answer: 2025-12-27

> Finished chain.


Reminder: Download your notebook to you local computer to save your work.